In [1]:
import os, pickle, random
import numpy as np
import torch

In [2]:
# change the current directory to the father directory of this file
os.chdir(os.path.dirname(os.path.dirname(os.path.realpath('evaluate2.ipynb'))))
print(os.getcwd())

/home/yangshuaiyu6791/RepresentAttack/inversion_sePoint


In [6]:
from modules.models import sPoints_net, ePoints_net, se_net
from DataLoader.dataloader import TrajDataset
emb_dim = 128
emb_model = 'neutraj'

# load data
with open("./data/{}/{}/train/train_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    train_embs = tmp_embs['embs']
with open("./data/{}/{}/train/train_grid".format(emb_model, emb_dim), "rb") as f:
    train_grids = pickle.load(f)

with open("./data/{}/{}/val/val_data".format(emb_model, emb_dim), "rb") as f:
    tmp_embs = pickle.load(f)
    val_embs = tmp_embs['embs']
with open("./data/{}/{}/val/val_grid".format(emb_model, emb_dim), "rb") as f:
    val_grids = pickle.load(f)
    
with open("./rdata/gps_seqs/traingps", "rb") as f:
    train_gps = pickle.load(f)
    train_gps = train_gps[0:60000]
with open("./rdata/gps_seqs/valgps", "rb") as f:
    val_gps = pickle.load(f)

In [7]:
# load pth
# delta = 0.001, emb_model = neutraj, embedding dim = emb_dim, hidden dim = 512

model = se_net(emb_dim, 512)
params = torch.load('./models/d0_001_train_sePoints_{}_{}_{}.pth'.format(emb_model, emb_dim, 512))
model.load_state_dict(params)

<All keys matched successfully>

In [12]:
# calculate the distance between two points with longitude and latitude
def distance(lat1, lon1, lat2, lon2):
    R = 6371.004    # km
    pi = 3.1415926
    dlat = (lat2 - lat1) * pi / 180
    dlon = (lon2 - lon1) * pi / 180
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(lat1 * pi / 180) * np.cos(lat2 * pi / 180) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

# transform the cell x,y to lon, lat
porto_lat_range = [40.953673,41.307945]
porto_lon_range = [-8.735152,-8.156309]
delta = 0.001

def grid2gps(grid):    # x -> lon, y -> lat
    grid_x = grid[0]; grid_y = grid[1]
    gps_x = porto_lon_range[0] + grid_x * delta
    gps_y = porto_lat_range[0] + grid_y * delta
    return [gps_x, gps_y]    # [lon, lat]

In [16]:
# choose 40 points by random
choice_list = np.random.choice(len(val_embs), 40, replace=False)
print([val_gps[i][0] for i in choice_list])    # lon, lat
choice_valgrids = [[val_grids[i][0][0], val_grids[i][0][1], val_grids[i][-1][0], val_grids[i][-1][1]] for i in choice_list]    # x, y
choice_valembs = [val_embs[i] for i in choice_list]
choice_valgps = [[val_gps[i][0], val_gps[i][-1]] for i in choice_list]    # [[lon, lat], [lon, lat]]
# get the prediction
model.cpu()
model.eval()
choice_predgrids = []
for i in range(len(choice_valembs)):    
    choice_valembs[i] = torch.Tensor(choice_valembs[i]).unsqueeze(0)
    choice_predgrids.append(model(choice_valembs[i]).cpu().detach().numpy()[0])

# for i in range(len(choice_predgrids)):
#     print(choice_predgrids[i], choice_valgrids[i], '\n')

# transform the prediction to lon, lat
choice_predgps = []
for i in range(len(choice_predgrids)):
    choice_predgps.append([grid2gps(choice_predgrids[i][0:2]), grid2gps(choice_predgrids[i][2:])])
print(choice_predgps)

[array([-8.608707, 41.157396]), array([-8.585658, 41.148504]), array([-8.58564 , 41.148486]), array([-8.604135, 41.14539 ]), array([-8.61102 , 41.147136]), array([-8.608752, 41.147712]), array([-8.60283 , 41.179617]), array([-8.663715, 41.177187]), array([-8.612595, 41.146101]), array([-8.609895, 41.15304 ]), array([-8.614836, 41.123799]), array([-8.600022, 41.182668]), array([-8.615484, 41.140962]), array([-8.656803, 41.161482]), array([-8.585658, 41.14854 ]), array([-8.628516, 41.155173]), array([-8.582355, 41.180427]), array([-8.64747 , 41.150781]), array([-8.597151, 41.156586]), array([-8.617545, 41.146164]), array([-8.612127, 41.172813]), array([-8.613225, 41.166792]), array([-8.619822, 41.148045]), array([-8.679978, 41.156523]), array([-8.628264, 41.157684]), array([-8.616798, 41.173317]), array([-8.627148, 41.15205 ]), array([-8.599959, 41.182641]), array([-8.612073, 41.148738]), array([-8.591355, 41.162742]), array([-8.611308, 41.149314]), array([-8.61984 , 41.148036]), array([

In [17]:
# calculate the distance between prediction and ground truth
start_dis = []; end_dis = []
for i in range(len(choice_predgps)):
    start_dis.append(distance(choice_predgps[i][0][1], choice_predgps[i][0][0], choice_valgps[i][0][1], choice_valgps[i][0][0]))
    end_dis.append(distance(choice_predgps[i][1][1], choice_predgps[i][1][0], choice_valgps[i][1][1], choice_valgps[i][1][0]))
print(start_dis)
print(end_dis)
print('average start distance: {} km'.format(np.mean(start_dis)))
print('average end distance: {} km'.format(np.mean(end_dis)))

with open("./results/{}_{}_{}_{}_dists".format(delta, emb_model, emb_dim, 512), "w") as f:
    f.write("choose 40 points by random\n")
    f.write("choice_list: {}\n".format(choice_list))
    f.write('average start distance: {} km'.format(np.mean(start_dis)))
    f.write('average end distance: {} km'.format(np.mean(end_dis)))
    f.write('\n')
    f.write('start distance: {}'.format(start_dis))
    f.write('\n')
    f.write('end distance: {}'.format(end_dis))

[1.1556665736933662, 1.3025588012191402, 1.2491463253433668, 1.5178337459363205, 1.1385400112732256, 1.4304361354618815, 1.1680665460251398, 0.8670156774991057, 1.187678967723948, 1.2103539290651935, 1.1101098393783035, 1.9371119943516233, 0.7053192000186634, 1.0143001567885026, 1.3604961371058981, 0.9223393359635894, 1.6887980416708999, 2.014613526505739, 1.2816244067304474, 1.2026257690095108, 1.1951756499486828, 0.7635312464774681, 1.1925586857079336, 0.762339778696144, 0.9299009220177853, 0.9003724654448372, 0.8521648543708084, 1.3509577216055118, 1.3597892451169122, 1.4957035512538424, 1.2158587943973835, 1.3257339472080447, 1.1144338656846722, 1.7285282228746075, 1.0944373258860645, 1.156271845603011, 0.9675516266569364, 1.5439043505252394, 1.4672237300243889, 0.9731711672317099]
[2.178006801287843, 2.3299095960768628, 2.3260079431052323, 2.8183759933035133, 2.0810399441980665, 2.5891499742248643, 2.650889781707873, 2.283875761139316, 2.0239978629315405, 2.2235215949991556, 1.932

In [18]:
# if you want, plot the prediction and ground truth on the map
import folium
my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=12)
choice = np.random.choice(len(choice_predgps))
print(choice_predgps[choice], choice_valgps[choice])

folium.CircleMarker(location=[choice_predgps[choice][0][1], choice_predgps[choice][0][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_predgps[choice][1][1], choice_predgps[choice][1][0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_valgps[choice][0][1], choice_valgps[choice][0][0]], radius=5, color='red', fill=True).add_to(my_map)
folium.CircleMarker(location=[choice_valgps[choice][1][1], choice_valgps[choice][1][0]], radius=5, color='red', fill=True).add_to(my_map)

# plot the real trajectory
gps_val = val_gps[choice_list[choice]][:, [1, 0]]
folium.PolyLine(gps_val, color="red", weight=2.5, opacity=1).add_to(my_map)

my_map   # show the map

[[-8.620727630187988, 41.16288701977539], [-8.674248366882324, 41.13916958203125]] [array([-8.613225, 41.166792]), array([-8.670168, 41.156262])]


: 